Playing games with Spuds

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [3]:
const ability_none = 999
const ability_imitative = 10
const ability_reciprocating = 20
const ability_romantic = 30
const ability_bibliophile = 40
const ability_melodious = 50
const ability_iconoclast = 90
const ability_intoxicating = 100
const ability_otaku = 110
const ability_sincere = 120
const ability_cowardly = 130


ability_name = Dict(
    ability_imitative => "Imitative*",
    ability_reciprocating => "Reciprocating*",
    ability_romantic => "Romantic*",
    ability_bibliophile => "Bibliophile*",
    ability_melodious => "Melodious*",
    ability_iconoclast => "Iconoclast*",
    ability_intoxicating => "Intoxicating*",
    ability_otaku => "Otaku*",
    ability_sincere => "Sincere*",
    ability_cowardly => "Cowardly*",
)

function fx_imitative(a::Spud, b::Spud)
    if b.h > a.h
        a = Spud(a.name, b.h, a.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_reciprocating(a::Spud, b::Spud)
    if b.f > a.f
        a = Spud(a.name, a.h, b.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_romantic(a::Spud, b::Spud)
    if b.p > a.l
        a = Spud(a.name, a.h, a.f, b.p, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_bibliophile(a::Spud, b::Spud)
    if b.r > a.l
        a = Spud(a.name, a.h, a.f, b.r, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_melodious(a::Spud, b::Spud)
    if b.p > b.r && b.p > b.s
        a = Spud(a.name, a.h, a.f, a.r, a.p + 3, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_iconoclast(a::Spud, b::Spud)
    v = sum([b.h,b.f,b.l,b.p,b.r,b.s] .== MXV)
    if v >= 3
        aa = [a.h, a.f, a.l, a.p, a.r, a.s]
        aa2 = map(x -> div(x, 4), aa)
        r = aa .+ aa2
        a = Spud(a.name, r[1], r[2], r[3], r[4], r[5], r[6], a.a1)
    end
    return [a, b]
end

function fx_intoxicating(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.f, b.a1)
    return [a, b]
end

function fx_otaku(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.h, b.s, b.a1)
    return [a, b]
end


function fx_sincere(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f + 2, b.f + 2, b.p, b.r, b.s, b.a1)
    return [a, b]
end

function fx_cowardly(a::Spud, b::Spud)
    if (a.p < b.p) || (a.r < b.r) || (a.s < b.s)
        a = Spud(a.name, a.h + 2, a.f - 2, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

ability_fx = Dict(
    ability_imitative => fx_imitative,
    ability_reciprocating => fx_reciprocating,
    ability_romantic => fx_romantic,
    ability_bibliophile => fx_bibliophile,
    ability_melodious => fx_melodious,
    ability_iconoclast => fx_iconoclast,
    ability_intoxicating => fx_intoxicating,
    ability_otaku => fx_otaku,
    ability_sincere => fx_sincere,
    ability_cowardly => fx_cowardly,
)

Dict{Int64, Function} with 10 entries:
  50  => fx_melodious
  20  => fx_reciprocating
  110 => fx_otaku
  10  => fx_imitative
  90  => fx_iconoclast
  30  => fx_romantic
  120 => fx_sincere
  130 => fx_cowardly
  40  => fx_bibliophile
  100 => fx_intoxicating

In [4]:
#const MXS = 100
const MXV = 9
const MNV = 1

# function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
#     #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
#     return h*(f+l) + f*(p+r+s)
# end

# function cost(a::Spud)::Int64
#     if a.a1 == ability_none
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s)
#     else
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s) + ability_cost[a.a1](a)
#     end
# end

1

In [5]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [6]:


function apply_ability(a::Spud, b::Spud)::Array{Spud}
    if a.a1 == ability_none
        return [a, b]
    else
        return ability_fx[a.a1](a, b)
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, utb)
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    end
end

eval_battle (generic function with 1 method)

In [7]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)


const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat(ability::Int64 = ability_none)::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    if ability == ability_none
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
        vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
        vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
        vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
        vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
        vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    else
        adj = ability_name[ability]
    end
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6], ability_none)
end

random_name_and_stat (generic function with 2 methods)

In [8]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat(a.a1)
    for ii in 1:n_tries
        b = random_name_and_stat(a.a1)
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s, a.a1)
end

rand_rename (generic function with 2 methods)

In [9]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

In [10]:
function sample_library(ss_prob, as, n_init = 1000000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for a1 in as
        for h in hrange
            for f in frange
                for l in lrange
                    for p in prange
                        for r in rrange
                            for s in srange
                                if rand() <= ss_prob
                                    ff = Spud("", h,f,l,p,r,s,a1)
                                    ff_i += 1
                                    ffs[ff_i] = ff
                                end
                            end
                        end                        
                    end                        
                end
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

sample_library (generic function with 2 methods)

In [11]:
struct NashEquilibrium2p
    w1::Array{Float64}
    w2::Array{Float64}
    v::Float64
end


function payoffs2(set1::Array{Spud}, set2::Array{Spud})::Array{Int64}
    n1 = length(set1)
    n2 = length(set2)
    payoffs = Array{Int64}(undef, (n1, n2))
    for i in 1:n1
        for j in 1:n2
            payoffs[i,j] = eval_battle(set1[i], set2[j])
        end
    end
    return payoffs
end

function ffp2(set1::Array{Spud}, set2::Array{Spud}, nits::Int64)::NashEquilibrium2p
    n1 = length(set1)
    n2 = length(set2)
    payoffs = payoffs2(set1, set2)
    v1 = [1 for i in 1:n1]
    v2 = [1 for i in 1:n2]
    s1 = [sum(payoffs[i, :] .* v2) for i in 1:n1]
    s2 = [sum(payoffs[:, i] .* v1) for i in 1:n2]
    for it in 1:nits
        cands1 = findall(s1 .== maximum(s1))
        i1 = rand(cands1)
        cands2 = findall(s2 .== minimum(s2))
        i2 = rand(cands2)
        v1[i1] += 1
        s2 = s2 .+ payoffs[i1, :]
        v2[i2] += 1
        s1 = s1 .+ payoffs[:, i2]
    end
    w1 = v1./sum(v1)
    w2 = v2./sum(v2)
    val = sum([payoffs[i, j] * w1[i] * w2[j] for i in 1:n1 for j in 1:n2])
    return NashEquilibrium2p(w1, w2, val)
end

function ffp(nash_env, nits)
    n_nash = length(nash_env)
    i_lose = Array{Int}(undef, (n_nash, n_nash))
    n_lose = Array{Int}(undef, n_nash)
    for i in 1:n_nash
        n_lose[i] = 0
        ff = nash_env[i]
        for j in 1:n_nash
            if eval_battle(ff, nash_env[j]) ==-1
                n_lose[i] += 1
                i_lose[i, n_lose[i]] = j
            end
        end
    end
    counts = [0 for i in 1:n_nash]
    wins = [0 for i in 1:n_nash]
    for i in 1:n_nash
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    for iter in 1:nits
        wc = 2 .* wins .+ counts
        ind_winners = findall(wc .== maximum(wc))
        i = rand(ind_winners)
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    return counts
end

ffp (generic function with 1 method)

## Battle code

In [12]:
mutable struct BattleView
    round::Int64
    own::Array{Spud}
    owncount::Array{Int64}
    deployed::Array{Bool}
    obs::Array{Spud}
    obscount::Array{Int64}
end

mutable struct AiPlayer
    genteam::Function
    pick::Function
    create::Function
end

In [13]:
n_copy = 3

function initialize_game(team1::Array{Spud}, team2::Array{Spud})::Array{BattleView}
    teams = [team1, team2]
    bvs = Array{BattleView}(undef, 0)
    for team in teams
        bv = BattleView(0, team, [n_copy for i in 1:length(team)], [false for i in 1:length(team)], [], [])
        append!(bvs, [bv])
    end
    return bvs
end

function observe_fighters(game::Array{BattleView}, choices::Array{Int64})::Array{BattleView}
    for i in 1:2
        j = 3- i
        if choices[i] != 0
            game[i].deployed[choices[i]] = true
            ff = game[i].own[choices[i]]
            if ff in game[j].obs
                0
            else
                game[j].obs = append!(game[j].obs, [ff])
                game[j].obscount = append!(game[j].obscount, [game[i].owncount[choices[i]]])
            end
        end
    end
    return game
end

function modify_fighter_counts(game::Array{BattleView}, choices::Array{Int64}, mod::Array{Int64})::Array{BattleView}
    for i in 1:2
        j = 3- i
        if choices[i] != 0
            game[i].owncount[choices[i]] = game[i].owncount[choices[i]] + mod[i]
            ff = game[i].own[choices[i]]
            if ff in game[j].obs
                ind = findall(x -> x==ff, game[j].obs)[1]
                game[j].obscount[ind] = game[i].owncount[choices[i]]
            else
                0
            end
        end
    end
    return game
end

function play_combat(game::Array{BattleView}, choices::Array{Int64}, verbose::Bool = false)::Array{BattleView}
    round = game[1].round + 1
    fighters = [game[i].own[choices[i]] for i in 1:2]
    for i in 1:2
        @assert game[i].owncount[choices[i]] >= 0
    end
    if verbose
        print("Round ")
        print(round)
        print(": #A")
        print(choices[1])
        print(" ")
        print(fighters[1])
        print(" vs #B")
        print(choices[2])
        print(" ")
        println(fighters[2])
    end
    outcome = eval_battle(fighters[1], fighters[2])
    if verbose
        if outcome == 0
            println("Tied!")
        else
            if outcome == 1
                print(fighters[1])
            end
            if outcome == -1
                print(fighters[2])
            end
            println(" wins!")
        end
    end
    game = observe_fighters(game, choices)
    if outcome > -1
        game = modify_fighter_counts(game, choices, [0, -1])
    end
    if outcome < 1
        game = modify_fighter_counts(game, choices, [-1, 0])
    end
    for i in 1:2
        game[i].round = round
    end
    return game
end

function play_add_spud(game::Array{BattleView}, player::Int64, ff::Spud, verbose::Bool = false)::Array{BattleView}
    game[player].own = append!(game[player].own, [ff])
    game[player].owncount = append!(game[player].owncount, [n_copy])
    game[player].deployed = append!(game[player].deployed, [false])
    if verbose
        print("Player ")
        print(player)
        print(" adds #")
        print(["A", "B"][player])
        print(length(game[player].own))
        print(" ")
        println(ff)
    end
    return game
end

function play_is_game_over(game::Array{BattleView})::Bool
    for bv in game
        if sum(bv.owncount) == 0
            return true
        end
    end
    return false
end

play_is_game_over (generic function with 1 method)

In [14]:
function generate_rand_team(env, counts, nteam)
    cc = cumsum(counts)
    team = Array{Spud}(undef, nteam)
    for i in 1:nteam
        tmp = rand(1:sum(counts))
        ind = sum(cc .< tmp)+1
        team[i] = env[ind]
    end
    return team
end

function ai0_pick(a::BattleView)::Int64
    inds = findall(a.owncount .> 0)
    return rand(inds)
end

function ai0_create(a::BattleView)::Spud
    return generate_rand_team(nash_env, counts, 1)[1]
end

function ai1_create(a::BattleView)::Spud
    ops = a.obs[a.obscount .> 0]
    ebs = [eval_battle_list(ff, ops) for ff in nash_env]
    cands = nash_env[ebs .== maximum(ebs)]
    return rand(cands)
end

function ai2_create(a::BattleView)::Spud
    ops = a.obs[a.obscount .> 0]
    ebs = [eval_battle_list(ff, ops) for ff in nash_env]
    if maximum(ebs) == length(ops)
        cands = nash_env[ebs .== maximum(ebs)]
        return rand(cands)
    else
        ebs2 = [eval_battle_list(ff, ops) for ff in lib]
        if maximum(ebs2) > maximum(ebs)
            cands = lib[ebs2 .== maximum(ebs2)]
            return rand_rename(rand(cands))
        else
            cands = nash_env[ebs .== maximum(ebs)]
            return rand(cands)
        end
    end
end


ai2_create (generic function with 1 method)

In [15]:
# picks Spud most likely to win
function ai_greedy_pick(a::BattleView)::Int64
    if length(a.obs[a.obscount .> 0]) == 0
        return rand(findall(a.owncount .> 0))
    end
    res = ffp2(a.own[a.owncount .> 0], a.obs[a.obscount .> 0], 100)
    ind = findall(a.owncount .> 0)[rand(findall(res.w1 .== maximum(res.w1)))]
    return ind
end

ai_greedy_pick (generic function with 1 method)

In [16]:
# picks from spuds already deployed
function ai_conservative_pick(a::BattleView)::Int64
    own = a.own[a.owncount .> 0 .&& a.deployed]
    owninds = findall(a.owncount .> 0 .&& a.deployed)
    if length(own) == 0
        # pick greedy
        return ai_greedy_pick(a)
    end
    ops = a.obs[a.obscount .> 0]
    res = ffp2(own, ops, 100)
    ind = owninds[rand(findall(res.w1 .== maximum(res.w1)))]
    return ind
end

ai_conservative_pick (generic function with 1 method)

In [17]:
# picks Spud based on 1-round nash
function ai_myopic_pick(a::BattleView)::Int64
    if length(a.obs[a.obscount .> 0]) == 0
        return rand(findall(a.owncount .> 0))
    end
    res = ffp2(a.own[a.owncount .> 0], a.obs[a.obscount .> 0], 100)
    cc = cumsum(res.w1)
    ind = sum(cc .< rand()) + 1
    return findall(a.owncount .> 0)[ind]
end

ai_myopic_pick (generic function with 1 method)

In [18]:
creation_rounds = [5, 10, 15]

function play_game(players::Array{AiPlayer}, verbose::Bool = true)::Int64
    ai1 = players[1]
    ai2 = players[2]
    ais = players
    team1 = ai1.genteam()
    team2 = ai2.genteam()
    game = initialize_game(team1, team2)
    while !play_is_game_over(game)
        # play a round
        choice1 = ai1.pick(game[1])
        choice2 = ai2.pick(game[2])
        game = play_combat(game, [choice1, choice2], verbose)
        if verbose
            for i in 1:2
                println(game[i].owncount)
            end
        end
        if game[1].round in creation_rounds
            for i in 1:2
                ff = ais[i].create(game[i])
                game = play_add_spud(game, i, ff, verbose)
            end
        end
    end
    if sum(game[1].owncount) == 0
        if sum(game[2].owncount) == 0
            return 0
        else
            return -1
        end
    end
    return 1
end

play_game (generic function with 2 methods)

In [19]:
# ai1 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai_greedy_pick, ai1_create)
# ai2 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai_greedy_pick, ai1_create)
# ais = [ai1, ai2]
# play_game(ais, true)

In [20]:
@time lib = sample_library(1.01, [999]);

  0.162828 seconds (62 allocations: 249.058 MiB, 11.38% gc time)


In [21]:
function filter_nondominated(as::Array{Spud})::Array{Spud}
    df = spuds_to_df(as)
    mat = Array{Int64}(undef, (length(as), 6))
    mat[:, 1] = df.h
    mat[:, 2] = df.f
    mat[:, 3] = df.l
    mat[:, 4] = df.p
    mat[:, 5] = df.r
    mat[:, 6] = df.s;
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:6
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

function spud_to_vec(a::Spud)::Array{Int64}
    return [a.h, a.f, a.l, a.p, a.r, a.s]
end

spud_to_vec (generic function with 1 method)

## Generating p.win data by calculation

### Tensor-cdf computations

In [22]:
function computeTCDF6(tpdf::Array{Int64})::Array{Int64}
    tcdf = 0 .* tpdf
    for i1 in 1:size(tpdf)[1]
        for i2 in 1:size(tpdf)[2]
            for i3 in 1:size(tpdf)[3]
                for i4 in 1:size(tpdf)[4]
                    for i5 in 1:size(tpdf)[5]
                        for i6 in 1:size(tpdf)[6]
                            tcdf[i1,i2,i3,i4,i5,i6] = sum(tpdf[1:i1,1:i2,1:i3,1:i4,1:i5,1:i6])
                        end
                    end
                end
            end
        end    
    end
    return tcdf
end

function integrate_cdf(tcdf::Array{Int64}, mins::Array{Int64}, maxes::Array{Int64}, fixed::Array{Int64} = Array{Int64}(undef, 0))::Int64
    if sum(mins .== maxes) >0
        return 0
    end
    if length(mins) == 0
        if minimum(fixed) == 0
            return 0
        else
            return tcdf[CartesianIndex(tuple(fixed...))]
        end
    end
    new_min = mins[1]
    new_max = maxes[1]
    new_fixed1 = append!(copy(fixed), [new_min])
    new_fixed2 = append!(copy(fixed), [new_max])
    ub = integrate_cdf(tcdf, mins[2:end], maxes[2:end], new_fixed2)
    if new_min <= 0
        return ub
    else
        lb = integrate_cdf(tcdf, mins[2:end], maxes[2:end], new_fixed1)
        return ub - lb
    end
end

integrate_cdf (generic function with 2 methods)

In [23]:
function computeTCDF6(tpdf::Array{Float64})::Array{Float64}
    tcdf = 0 .* tpdf
    for i1 in 1:size(tpdf)[1]
        for i2 in 1:size(tpdf)[2]
            for i3 in 1:size(tpdf)[3]
                for i4 in 1:size(tpdf)[4]
                    for i5 in 1:size(tpdf)[5]
                        for i6 in 1:size(tpdf)[6]
                            tcdf[i1,i2,i3,i4,i5,i6] = sum(tpdf[1:i1,1:i2,1:i3,1:i4,1:i5,1:i6])
                        end
                    end
                end
            end
        end    
    end
    return tcdf
end

function integrate_cdf(tcdf::Array{Float64}, mins::Array{Int64}, maxes::Array{Int64}, fixed::Array{Int64} = Array{Int64}(undef, 0))::Float64
    if sum(mins .== maxes) >0
        return 0
    end
    if length(mins) == 0
        if minimum(fixed) == 0
            return 0
        else
            return tcdf[CartesianIndex(tuple(fixed...))]
        end
    end
    new_min = mins[1]
    new_max = maxes[1]
    new_fixed1 = append!(copy(fixed), [new_min])
    new_fixed2 = append!(copy(fixed), [new_max])
    ub = integrate_cdf(tcdf, mins[2:end], maxes[2:end], new_fixed2)
    if new_min <= 0
        return ub
    else
        lb = integrate_cdf(tcdf, mins[2:end], maxes[2:end], new_fixed1)
        return ub - lb
    end
end

integrate_cdf (generic function with 4 methods)

In [63]:
fixed = []
mins = [1,2,3]
maxes = [2,4,6];
new_min = mins[1]
new_max = maxes[1]
new_fixed1 = append!(copy(fixed), [new_min])
println(new_fixed1)
new_fixed2 = append!(copy(fixed), [new_max])

Any[1]


1-element Vector{Any}:
 2

In [57]:
# tpdf = ones(Float64, (6,6,6,6,6,6));
# @time tcdf = computeTCDF6(tpdf);
# # 0.462401 seconds (105.26 k allocations: 664.380 MiB, 45.45% gc time)

  0.462401 seconds (105.26 k allocations: 664.380 MiB, 45.45% gc time)


In [58]:
# tpdf = ones(Int64, (6,6,6,6,6,6));
# @time tcdf = computeTCDF6(tpdf);
# # 0.491771 seconds (105.26 k allocations: 664.380 MiB, 49.74% gc time)

  0.491771 seconds (105.26 k allocations: 664.380 MiB, 49.74% gc time)


In [59]:
tpdf = ones(Int64, (9,9,9,9,9,9));
@time tcdf = computeTCDF6(tpdf);

In [27]:
# cdf([a,b]x[c,d]) = cdf(b, [c,d]) - cdf[a, [c,d]]
# cdf(b, [c,d]) = cdf(b,d) - cdf(b,c)


In [64]:
mins = [0,1,2,1,0,0]
maxes = [3,3,4,5,3,3]
println(prod(maxes .- mins))
integrate_cdf(tcdf, mins, maxes)

432


432

In [66]:
mins = [3,1,2,1,0,0]
maxes = [3,3,4,5,3,3]
integrate_cdf(convert(Array{Float64}, tcdf), mins, maxes)

0.0

### Computing p-win from a tensor-cdf

In [24]:
# spuds to min-max matrix
function spuds_to_mmm0(as::Array{Spud})::Array{Int64}
    df = spuds_to_df(as)
    mat = zeros(Int64, (length(as), 12))
    mat[:, 1] = df.h
    mat[:, 3] = df.f
    mat[:, 5] = df.l
    mat[:, 7] = df.p
    mat[:, 9] = df.r
    mat[:,11] = df.s;
    mat[:, 2] = df.h
    mat[:, 4] = df.f
    mat[:, 6] = df.l
    mat[:, 8] = df.p
    mat[:,10] = df.r
    mat[:,12] = df.s;
    return mat
end

function spuds_to_mmm(as::Array{Spud})::Array{Int64}
    mmm = spuds_to_mmm0(as);
    for j in [6,5,4,3,2,1]
        inds_used = zeros(Int64, size(mmm)[1]);
        mmm2 = Array{Int64}(undef, (size(mmm)[1], 12));
        n_mmm2 = 0;
        while minimum(inds_used) == 0
            ind_next = findall(inds_used .== 0)[1]
            v = mmm[ind_next, :];
            eq = ones(Int64, size(mmm)[1]);
            eq = eq .* (mmm[:, (2 * j -1)] .== mmm[:, (2 * j)])
            for k in 1:6
                if k != j
                    eq = eq .* (mmm[:, (2 * k -1)] .== v[(2 * k -1)])
                    eq = eq .* (mmm[:, (2 * k)] .== v[(2 * k)])
                end
            end
            mmsub = mmm[eq .== 1, :]
            minv = minimum(mmsub[:, 2*j])
            maxv = maximum(mmsub[:, 2*j])
            inds_used[eq .== 1] .= 1;
            if size(mmsub)[1] == (maxv - minv + 1)
                vnew = copy(v)
                vnew[2 * j - 1] = minv
                vnew[2 * j] = maxv
                n_mmm2 += 1
                mmm2[n_mmm2, :] = vnew
            else
                nsub = size(mmsub)[1]
                mmm2[(n_mmm2+1):(n_mmm2+nsub), :] = mmsub
                n_mmm2 += nsub
            end
        end
        mmm2 = mmm2[1:n_mmm2, :]
        mmm = mmm2;
    end
    return mmm
end

spuds_to_mmm (generic function with 1 method)

In [25]:
n_init = 10000
# generate a library where MNV = 1 and MXV = 3
ffs = Array{Spud}(undef, n_init)
ff_i = 0
hrange = 1:3
frange = 1:3
lrange = 1:3
prange = 1:3
rrange = 1:3
srange = 1:3
a1 = 999
for h in hrange
    for f in frange
        for l in lrange
            for p in prange
                for r in rrange
                    for s in srange
                        ff = Spud("", h,f,l,p,r,s,a1)
                        ff_i += 1
                        ffs[ff_i] = ff
                    end
                end                        
            end                        
        end
    end
end
ffs = ffs[1:ff_i]
libmax3 = ffs;

In [26]:
n_init = 10000
# generate a library where MNV = 1 and MXV = 3 (except for H, F where MXV = 4)
ffs = Array{Spud}(undef, n_init)
ff_i = 0
hrange = 1:4
frange = 1:4
lrange = 1:3
prange = 1:3
rrange = 1:3
srange = 1:3
a1 = 999
for h in hrange
    for f in frange
        for l in lrange
            for p in prange
                for r in rrange
                    for s in srange
                        ff = Spud("", h,f,l,p,r,s,a1)
                        ff_i += 1
                        ffs[ff_i] = ff
                    end
                end                        
            end                        
        end
    end
end
ffs = ffs[1:ff_i]
libmax4 = ffs;

In [27]:
n_init = 10000
# generate a library where MNV = 1 and MXV = 3 (except for H, F where MXV = 5)
ffs = Array{Spud}(undef, n_init)
ff_i = 0
hrange = 1:5
frange = 1:5
lrange = 1:3
prange = 1:3
rrange = 1:3
srange = 1:3
a1 = 999
for h in hrange
    for f in frange
        for l in lrange
            for p in prange
                for r in rrange
                    for s in srange
                        ff = Spud("", h,f,l,p,r,s,a1)
                        ff_i += 1
                        ffs[ff_i] = ff
                    end
                end                        
            end                        
        end
    end
end
ffs = ffs[1:ff_i]
libmax5 = ffs;

In [28]:
## case 1. H = F
ff = Spud("",2,2,2,2,2,2,999)
evs = [eval_battle(ff, gg) for gg in libmax3];
i_beat = libmax3[evs .== 1]
i_tie = libmax3[evs .== 0]
length(i_beat)
mmm_h_eq_f = spuds_to_mmm(i_beat);

In [29]:
## case 2. H = F+1
ff = Spud("",3,2,2,2,2,2,999)
evs = [eval_battle(ff, gg) for gg in libmax4];
i_beat = libmax4[evs .== 1]
i_tie = libmax4[evs .== 0]
length(i_beat)
mmm_h_gt_f = spuds_to_mmm(i_beat);

In [30]:
## case 3. H = F-1
ff = Spud("",2,3,2,2,2,2,999)
evs = [eval_battle(ff, gg) for gg in libmax4];
i_beat = libmax4[evs .== 1]
i_tie = libmax4[evs .== 0]
length(i_beat)
mmm_h_lt_f = spuds_to_mmm(i_beat);

In [31]:
## case 4. H > F+1
ff = Spud("",4,2,2,2,2,2,999)
evs = [eval_battle(ff, gg) for gg in libmax5];
i_beat = libmax5[evs .== 1]
i_tie = libmax5[evs .== 0]
length(i_beat)
mmm_h_mg_f = spuds_to_mmm(i_beat);

In [32]:
## case 5. H < F-1
ff = Spud("",2,4,2,2,2,2,999)
evs = [eval_battle(ff, gg) for gg in libmax5];
i_beat = libmax5[evs .== 1]
i_tie = libmax5[evs .== 0]
length(i_beat)
mmm_h_ml_f = spuds_to_mmm(i_beat);

In [33]:
function get_win_regions_vanilla_vs_vanilla(a::Spud)::Tuple{Array{Int64}, Array{Int64}}
    v = spud_to_vec(a)
    thres_s = Array{Array{Int64}}(undef, 6)

    ## case 1. H = F
    if v[1]==v[2]
        mmm = mmm_h_eq_f
        hf_thres = [-1, v[1]-1, v[1], MXV+1]    
    end

    ## case 2. H = F+1
    if v[1]==v[2]+1
        mmm = mmm_h_gt_f
        hf_thres = [-1, v[2]-1, v[2], v[1], MXV+1]
    end

    ## case 3. H = F-1
    if v[1]==v[2]-1
        mmm = mmm_h_lt_f
        hf_thres = [-1, v[1]-1, v[1], v[2], MXV+1]
    end

    ## case 4. H > F+1
    if v[1]> v[2]+1
        mmm = mmm_h_mg_f
        hf_thres = [-1, v[2]-1, v[2], v[1]-1, v[1], MXV+1]
    end

    ## case 5. H < F-1
    if v[1]< v[2]-1
        mmm = mmm_h_ml_f
        hf_thres = [-1, v[1]-1, v[1], v[2]-1, v[2], MXV+1]
    end


    thres_s[1] = hf_thres
    thres_s[2] = hf_thres
    for ii in 3:6
        thres_s[ii] = [-1, v[ii]-1, v[ii], MXV+1]
    end
    mins_raw = Array{Array{Int64}}(undef, 6)
    maxes_raw = Array{Array{Int64}}(undef, 6)
    inds_min = [1,3,5,7,9,11]
    inds_max = [2,4,6,8,10,12]
    for ii in 1:6
        mins_raw[ii] = thres_s[ii][mmm[:, inds_min[ii]]]
        maxes_raw[ii] = thres_s[ii][mmm[:, inds_max[ii]] .+ 1]
    end

    mins = reduce(hcat, mins_raw);
    maxes = reduce(hcat, maxes_raw);
    mins[mins .< 0] .= 0;
    mins[mins .> MXV] .= MXV;
    maxes[maxes .< 0] .= 0;
    maxes[maxes .> MXV] .= MXV;
    return (mins, maxes)
end

function total_evs_vanilla_vs_vanilla(tcdf::Array{Int64}, a::Spud)::Int64
    res = get_win_regions_vanilla_vs_vanilla(a)
    mins = res[1]
    maxes = res[2]
    sums = [integrate_cdf(tcdf, mins[ii, :], maxes[ii, :]) for ii in 1:size(mins)[1]];
    return (2 * sum(sums) - length(lib) + 1)
end

function total_evs_vanilla_vs_vanilla(tcdf::Array{Float64}, a::Spud)::Float64
    res = get_win_regions_vanilla_vs_vanilla(a)
    mins = res[1]
    maxes = res[2]
    sums = [integrate_cdf(tcdf, mins[ii, :], maxes[ii, :]) for ii in 1:size(mins)[1]];
    return (2 * sum(sums) - length(lib) + 1)
end


total_evs_vanilla_vs_vanilla (generic function with 2 methods)

### Unit-testing

In [34]:
# # TODO
# for iiiii in 1:1000
#     a = rand(lib)
#     v = spud_to_vec(a)
#     evs = [eval_battle(a, gg) for gg in lib];
#     i_beat = lib[evs .== 1];
#     i_lose = lib[evs .== -1];
#     #mean(evs)
#     res = get_win_regions_vanilla_vs_vanilla(a)
#     mins = res[1]
#     maxes = res[2]
#     for iiii in 1:1000
#         v2 = spud_to_vec(rand(i_lose))
#         min_cond = reduce((x,y) -> x .&& y, [v2[ii] .> mins[:,ii] for ii in 1:6])
#         max_cond = reduce((x,y) -> x .&& y, [v2[ii] .<= maxes[:,ii] for ii in 1:6])
#         if sum(min_cond .&& max_cond) != 0
#             println(v2)
#         end
#     end
#     for iiii in 1:1000
#         v2 = spud_to_vec(rand(i_beat))
#         min_cond = reduce((x,y) -> x .&& y, [v2[ii] .> mins[:,ii] for ii in 1:6])
#         max_cond = reduce((x,y) -> x .&& y, [v2[ii] .<= maxes[:,ii] for ii in 1:6])
#         if sum(min_cond .&& max_cond) != 1
#             println(v2)
#         end
#     end
# end

In [35]:
# for iiiii in 1:1000
#     a = rand(lib)
#     evs = [eval_battle(a, gg) for gg in lib];
#     if sum(evs) != total_evs_vanilla_vs_vanilla(tcdf, a)
#         println(a)
#     end
# end

### generate p.win data and iterate

In [ ]:
tpdf = ones(Int64, (9,9,9,9,9,9));
@time tcdf = computeTCDF6(tpdf);

In [229]:
@time scores0 = [total_evs_vanilla_vs_vanilla(tcdf, a) for a in lib]./length(lib);

423.404060 seconds (7.03 G allocations: 468.896 GiB, 12.37% gc time, 0.03% compilation time)


Row,name,h,f,l,p,r,s,a1,ev
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64
1,,9,2,3,9,6,1,999,-0.312324
2,,2,9,5,3,6,7,999,0.267002
3,,4,1,6,1,9,8,999,-0.31984
4,,9,7,4,5,3,8,999,0.677656
5,,2,1,8,1,3,7,999,-0.655094
6,,1,9,3,5,7,4,999,0.0564729
7,,2,9,3,9,4,3,999,0.125199
8,,4,2,6,3,4,1,999,-0.554338
9,,3,4,9,4,2,3,999,-0.415485


In [264]:
wts = map(x -> exp(-abs(x)/10), scores0);
wts = wts./sum(wts) .* length(lib);
println(maximum(scores0[wts.>1]))
println(minimum(scores0[wts.>1]))


#### iterations

In [265]:
tpdf = permutedims(reshape(wts, (9,9,9,9,9,9)), (6,5,4,3,2,1));
@time tcdf = computeTCDF6(tpdf);

 31.076102 seconds (1.55 M allocations: 61.968 GiB, 21.72% gc time, 0.38% compilation time)


In [ ]:
@time scores1 = [total_evs_vanilla_vs_vanilla(tcdf, a) for a in lib]./length(lib);
wts = map(x -> exp(-abs(x)/10), scores1);
wts = wts./sum(wts) .* length(lib);
println(maximum(scores1[wts.>1]))
println(minimum(scores1[wts.>1]))

In [ ]:
tpdf = permutedims(reshape(wts, (9,9,9,9,9,9)), (6,5,4,3,2,1));
@time tcdf = computeTCDF6(tpdf);
@time scores2 = [total_evs_vanilla_vs_vanilla(tcdf, a) for a in lib]./length(lib);
wts = map(x -> exp(-abs(x)/10), scores2);
wts = wts./sum(wts) .* length(lib);
println(maximum(scores2[wts.>1]))
println(minimum(scores2[wts.>1]))

In [ ]:
allscores = [scores0]
scores1 = scores0
for iiiii in 1:10
    wts = map(x -> exp(-abs(x)/10), scores1)
    wts = wts./sum(wts) .* length(lib);
    tpdf = permutedims(reshape(wts, (9,9,9,9,9,9)), (6,5,4,3,2,1));
    tcdf = computeTCDF6(tpdf);
    @time scores1 = [total_evs_vanilla_vs_vanilla(tcdf, a) for a in lib]./length(lib);
    append!(allscores, scores1)
end


In [ ]:
allscores = [scores0]
scores = scores0;
for i in 1:25
    wts = map(x -> exp(-300*abs(x)^2), scores)
    wts = wts./sum(wts) .* length(lib);
    tpdf = permutedims(reshape(wts, (9,9,9,9,9,9)), (6,5,4,3,2,1));
    tcdf = computeTCDF6(tpdf);
    @time scores = [total_evs_vanilla_vs_vanilla(tcdf, a) for a in lib]./length(lib);
    append!(allscores, [scores]);
    println(sum(wts .> 1))
    println(maximum(scores[wts.>1]))
    println(minimum(scores[wts.>1]))
end

In [ ]:
iis = [rand(1:length(lib)) for i in 1:100];
df = spuds_to_df(lib[iis]);
df[:, :ev0] = scores0[iis];
df[:, :ev] = scores[iis];
df[:, :w] = wts[iis];
df[df.w .> 0.5, :]

In [ ]:
# iscores = map(x -> convert(Int64, floor(x)), scores .* 100 .+ 100);
# iscores = permutedims(reshape(iscores, (9,9,9,9,9,9)), (6,5,4,3,2,1));
# using MAT
# matwrite("scores_sq100.mat", Dict("scores" => iscores); compress = true)

## Checking nash equilibria of evolved sets

In [34]:
using MAT
res = matread("scores_sq100.mat");

In [35]:
scores = res["scores"];

In [36]:
using JuMP
using HiGHS

In [42]:
inds = findall(scores .> 99 .&& scores .<= 100);

In [43]:
length(inds)

2200

In [44]:
nash_env = [Spud("",v[1],v[2],v[3],v[4],v[5],v[6],999) for v in inds];

In [40]:
#@time counts = ffp(nash_env, 20000);

In [41]:
#nash_env = nash_env[counts .> 1];

In [45]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

get_payoffs (generic function with 1 method)

In [46]:
payoffs = get_payoffs(nash_env);

In [47]:
n_nash = size(payoffs)[1]
vector_model = Model(HiGHS.Optimizer)
sumv = zeros(Int64, (1, 2 * n_nash + 1))
sumv[1:n_nash] .= 1
A = hcat(payoffs, Matrix(1I,n_nash, n_nash), -ones(Int64, n_nash, 1))
A = vcat(A, sumv);
b = zeros(Int64, n_nash + 1);
b[1:n_nash] .= -1
b[n_nash+1] = 1
c = zeros(Int64, 2 * n_nash + 1);
c[2 * n_nash + 1] = 1;
@variable(vector_model, x[1:(2*n_nash+1)] >= 0);
@constraint(vector_model, A * x .== b);
@objective(vector_model, Min, c' * x)
vector_model

A JuMP Model
Minimization problem with:
Variables: 4401
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2201 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 4401 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: x

In [48]:
@time optimize!(vector_model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2201 rows, 4401 cols, 4844400 nonzeros
2201 rows, 4401 cols, 4844400 nonzeros
Presolve : Reductions: rows 2201(-0); columns 4401(-0); elements 4844400(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2201(2201) 1s
       1727     9.9051435928e-01 Pr: 1497(231.345) 8s
       2574     9.9430571926e-01 Pr: 1323(7726.67) 14s
       3053     9.9563793475e-01 Pr: 1266(434.045) 20s
       3514     9.9662663274e-01 Pr: 1164(1515.2); Du: 0(8.44604e-08) 26s
       3961     9.9742939964e-01 Pr: 1082(69.1566) 32s
       4353     9.9797328087e-01 Pr: 1126(415.808) 38s
       4765     9.9844753692e-01 Pr: 1058(82.0721) 44s
       5106     9.9871365942e-01 Pr: 1087(118.378) 51s
       5518     9.9898720185e-01 Pr: 1131(591.634) 57s
       5806     9.9921008960e-01 Pr

In [49]:
w_lp = [value(x[i]) for i in 1:n_nash]
w_lp = w_lp./sum(w_lp);

In [50]:
sum(w_lp .> 0)

965

In [52]:
inds = findall(scores .> 95 .&& scores .<= 100);
env = [Spud("",v[1],v[2],v[3],v[4],v[5],v[6],999) for v in inds];
length(env)

11114

In [53]:
w = w_lp;
@time evs = [eval_battle_list2(ff, nash_env[w .> 0], w[w.> 0]) for ff in env];

  9.140737 seconds (157.63 M allocations: 14.463 GiB, 24.62% gc time, 1.49% compilation time)


In [54]:
maximum(evs)

0.015445719040705653

In [55]:
df = spuds_to_df(nash_env[w .> 0]);
df[:, :w] = w[w .> 0];
print(df[sortperm(-df.w)[1:20], :])

20×9 DataFrame
 Row │ name    h      f      l      p      r      s      a1     w          
     │ String  Int64  Int64  Int64  Int64  Int64  Int64  Int64  Float64    
─────┼─────────────────────────────────────────────────────────────────────
   1 │             9      3      3      9      5      9    999  0.0045518
   2 │             4      5      1      2      8      9    999  0.00450398
   3 │             7      1      5      9      3      9    999  0.00440355
   4 │             5      5      9      6      5      1    999  0.00412647
   5 │             7      2      7      3      1      1    999  0.00393954
   6 │             2      6      3      8      3      7    999  0.0038817
   7 │             9      5      1      9      8      2    999  0.00382939
   8 │             1      7      1      3      7      7    999  0.00373159
   9 │             8      3      4      5      3      8    999  0.00368775
  10 │             1      6      2      7      6      6    999  0.00355734
  11 │   

In [56]:
old_nash_env = nash_env;

In [64]:
nash_env = env[evs .> -0.01];
length(nash_env)

2971

In [65]:
@time payoffs = get_payoffs(nash_env);

  7.201042 seconds (129.60 M allocations: 11.300 GiB, 24.52% gc time)


In [66]:
n_nash = size(payoffs)[1]
vector_model = Model(HiGHS.Optimizer)
sumv = zeros(Int64, (1, 2 * n_nash + 1))
sumv[1:n_nash] .= 1
A = hcat(payoffs, Matrix(1I,n_nash, n_nash), -ones(Int64, n_nash, 1))
A = vcat(A, sumv);
b = zeros(Int64, n_nash + 1);
b[1:n_nash] .= -1
b[n_nash+1] = 1
c = zeros(Int64, 2 * n_nash + 1);
c[2 * n_nash + 1] = 1;
@variable(vector_model, x[1:(2*n_nash+1)] >= 0);
@constraint(vector_model, A * x .== b);
@objective(vector_model, Min, c' * x)
vector_model

A JuMP Model
Minimization problem with:
Variables: 5943
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2972 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 5943 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: x

In [67]:
@time optimize!(vector_model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2972 rows, 5943 cols, 8832783 nonzeros
2972 rows, 5943 cols, 8832783 nonzeros
Presolve : Reductions: rows 2972(-0); columns 5943(-0); elements 8832783(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2972(2972) 2s
       1008     9.7897640037e-01 Pr: 2508(2624.04) 7s
       1801     9.8663929084e-01 Pr: 2164(886.994) 14s
       2439     9.9001978441e-01 Pr: 1997(5401.88) 20s
       2902     9.9188835381e-01 Pr: 1871(504.106); Du: 0(6.61695e-08) 27s
       3406     9.9355645491e-01 Pr: 1798(444.24) 34s
       3781     9.9450422679e-01 Pr: 1683(382.179) 41s
       4097     9.9511215912e-01 Pr: 1646(1514.02) 48s
       4377     9.9561525754e-01 Pr: 1635(5006.46) 55s
       4723     9.9612979958e-01 Pr: 1654(2481.88) 63s
       5037     9.9653426291e-01 Pr

In [68]:
w_lp = [value(x[i]) for i in 1:n_nash]
w_lp = w_lp./sum(w_lp);
w = w_lp;
@time evs = [eval_battle_list2(ff, nash_env[w .> 0], w[w.> 0]) for ff in env];
maximum(evs)

 11.472737 seconds (200.69 M allocations: 18.406 GiB, 24.70% gc time, 0.24% compilation time)


0.006356875869242094

In [69]:
sum(w .> 0)

1231

In [70]:
df = spuds_to_df(nash_env[w .> 0]);
df[:, :w] = w[w .> 0];
print(df[sortperm(-df.w)[1:20], :])

20×9 DataFrame
 Row │ name    h      f      l      p      r      s      a1     w          
     │ String  Int64  Int64  Int64  Int64  Int64  Int64  Int64  Float64    
─────┼─────────────────────────────────────────────────────────────────────
   1 │             7      1      5      9      3      9    999  0.00416185
   2 │             9      3      3      9      5      9    999  0.00411995
   3 │             7      2      7      3      1      1    999  0.00364905
   4 │             3      5      2      9      6      7    999  0.00347259
   5 │             1      7      1      3      7      7    999  0.0033478
   6 │             2      6      3      8      3      7    999  0.00328024
   7 │             9      4      2      4      3      9    999  0.00323382
   8 │             7      1      7      2      2      2    999  0.00308478
   9 │             4      5      1      2      8      9    999  0.00306474
  10 │             5      8      8      1      4      3    999  0.00305725
  11 │  

In [73]:
df[:, :name] .= "nn"
CSV.write("temp_desL_sq100.csv", df)

"temp_desL_sq100.csv"

## Generating p.win data by brute force

In [81]:
length(lib)

531441

Round 1 of filtering

In [88]:
ref = [rand(lib) for i in 1:100]
@time scores = [eval_battle_list(ff, ref) for ff in lib];
scores = scores./length(ref);
scores_rep0 = scores;

 40.534909 seconds (769.61 M allocations: 66.847 GiB, 23.68% gc time, 0.06% compilation time)


In [89]:
ref = [rand(lib) for i in 1:100]
@time scores = [eval_battle_list(ff, ref) for ff in lib];
scores = scores./length(ref);
scores_rep1 = scores;

 41.490495 seconds (770.64 M allocations: 66.924 GiB, 22.88% gc time, 0.07% compilation time)


In [90]:
maximum(map(abs, scores_rep1 - scores_rep0))

0.44

In [91]:
scores = (scores_rep0 .+ scores_rep1) .* 0.5;

In [92]:
# filter library step 1
lib2 = lib[scores .>= - 0.5 .&& scores .<=  0.5]
length(lib2)

327125

Round 2 of filtering

In [87]:
ref = [rand(lib) for i in 1:500]
@time scores = [eval_battle_list(ff, ref) for ff in lib2];
scores = scores./length(ref);
scores_rep0 = scores;

 76.811622 seconds (1.44 G allocations: 124.878 GiB, 19.24% gc time, 0.03% compilation time)


In [88]:
ref = [rand(lib) for i in 1:500]
@time scores = [eval_battle_list(ff, ref) for ff in lib2];
scores = scores./length(ref);
scores_rep1 = scores;

 80.277977 seconds (1.44 G allocations: 125.079 GiB, 18.60% gc time, 0.03% compilation time)


In [89]:
maximum(map(abs, scores_rep1 - scores_rep0))

0.21999999999999997

In [90]:
scores = (scores_rep0 .+ scores_rep1) .* 0.5;

In [91]:
lib3 = lib2[scores .>= - 0.2 .&& scores .<= 0.2]
length(lib3)

138759

Round 3

In [92]:
ref = [rand(lib) for i in 1:1000]
@time scores = [eval_battle_list(ff, ref) for ff in lib3];
scores = scores./length(ref);
scores_rep0 = scores;

111.213163 seconds (2.02 G allocations: 175.472 GiB, 18.40% gc time, 0.02% compilation time)


In [93]:
ref = [rand(lib) for i in 1:1000]
@time scores = [eval_battle_list(ff, ref) for ff in lib3];
scores = scores./length(ref);
scores_rep1 = scores;

112.697692 seconds (2.02 G allocations: 175.081 GiB, 17.65% gc time, 0.02% compilation time)


In [94]:
maximum(map(abs, scores_rep1 - scores_rep0))

0.15600000000000003

In [95]:
scores = (scores_rep0 .+ scores_rep1) .* 0.5;

In [96]:
lib4 = lib3[scores .>= - 0.1 .&& scores .<= 0.1]
length(lib4)

70351

Round 4

In [98]:
ref = [rand(lib) for i in 1:2000]
@time scores = [eval_battle_list(ff, ref) for ff in lib4];
scores = scores./length(ref);
scores_rep0 = scores;

106.849612 seconds (2.05 G allocations: 177.827 GiB, 17.55% gc time, 0.02% compilation time)


In [99]:
ref = [rand(lib) for i in 1:2000]
@time scores = [eval_battle_list(ff, ref) for ff in lib4];
scores = scores./length(ref);
scores_rep1 = scores;

105.142988 seconds (2.05 G allocations: 177.716 GiB, 17.51% gc time, 0.02% compilation time)


In [100]:
scores = (scores_rep0 .+ scores_rep1) .* 0.5;

In [101]:
maximum(map(abs, scores_rep1 - scores_rep0))

0.135

In [102]:
lib5 = lib4[scores .>= - 0.05 .&& scores .<= 0.05]
length(lib5)

34520

## Evaluate internal fitness

In [93]:
df = spuds_to_df(lib2);

In [94]:
libsub = [rand(lib) for i in 1:2000];
@time evs = [eval_battle_list(ff, libsub) for ff in lib2];
w = evs./length(libsub);
df[:, :vsAll] = w

488.959264 seconds (9.53 G allocations: 826.715 GiB, 20.27% gc time, 0.00% compilation time)


327125-element Vector{Float64}:
 -0.531
 -0.523
 -0.516
 -0.508
 -0.502
 -0.514
 -0.503
 -0.493
 -0.512
 -0.508
 -0.5
 -0.495
 -0.488
  ⋮
  0.416
  0.4305
  0.444
  0.449
  0.409
  0.42
  0.434
  0.415
  0.421
  0.423
  0.426
  0.429

In [95]:
libsub = [rand(lib5) for i in 1:2000];
@time evs = [eval_battle_list(ff, libsub) for ff in lib2];
w = evs./length(libsub);
df[:, :vsSel] = w;

490.564353 seconds (9.59 G allocations: 831.670 GiB, 19.31% gc time, 0.01% compilation time)


In [96]:
CSV.write("temp_desL_evs_vs_zeroset.csv", df)

"temp_desL_evs_vs_zeroset.csv"

In [103]:
ff = Spud("", 3, 7, 1, 1, 9, 9, 999)
eval_battle_list(ff, lib5)/length(lib5)

0.5902085747392816

In [104]:
ff = Spud("", 9, 7, 2, 2, 2, 4, 999)
eval_battle_list(ff, lib5)/length(lib5)

0.583082271147161

In [109]:
ff = Spud("", 9, 7, 1, 1, 1, 1, 999)
eval_battle_list(ff, lib5)/length(lib5)

0.3447276940903824

## Use regression coeffs to predict Zero-Zero set

In [49]:
df = CSV.read("temp_desL_init_zerosubset.csv", DataFrame);
df[!, :name] .= "noname"
lib5 = df_to_spuds(df);

In [50]:
ff = rand(lib5)
ev = eval_battle_list(ff, lib)/length(lib)

-0.017145835567824085

In [51]:
using MAT

In [52]:
vars = matread("temp_desL_regress.mat")

Dict{String, Any} with 3 entries:
  "b0" => -0.528944
  "b2" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.00190…
  "b1" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0141574 0.0152…

In [53]:
b0 = vars["b0"];
b1 = vars["b1"];
b2 = vars["b2"];

function predict_ev(a::Spud, b0::Float64, b1::Array{Float64}, b2::Array{Float64})::Float64
    s = b0
    x = [a.h, a.f, a.l, a.p, a.r, a.s]
    for i in 1:6
        s += b1[i, x[i] + 1]
    end
    for i in 1:5
        for j in (i+1):6
            s += b2[i,j,x[i]+1,x[j]+1]
        end
    end
    return s
end

predict_ev (generic function with 2 methods)

In [86]:
ff = rand(lib)
println(ff)
predict_ev(ff,b0,b1,b2)

Spud("", 4, 1, 5, 6, 8, 3, 999)


-0.3595682845475866

In [87]:
@time ev = eval_battle_list(ff, lib5)/length(lib5)

  0.041915 seconds (503.35 k allocations: 44.723 MiB, 38.19% gc time)


-0.8001738122827347

## Evaluating cost tolerance

In Spud-Part system, some excess cost will be introduced due to limitation of finite part set.  How much excess cost can be tolerated?  This depends on how well Spuds with slightly lower value do against Spuds with maximum value.

In [23]:
using MAT
scores = matread("scores_sq300.mat")["scores"];

In [24]:
function get_spuds_with_value(scores::Array{Int64}, v::Int64)::Array{Spud}
    inds = findall(scores .== v)
    return map(x -> Spud("spud",x[1],x[2],x[3],x[4],x[5],x[6],ability_none), inds)
end


get_spuds_with_value (generic function with 1 method)

In [32]:
setA = get_spuds_with_value(scores, 100)
setB = get_spuds_with_value(scores, 90)
@time rf = ffp2(setA, setB, 100000);
rf.v

  4.389596 seconds (72.02 M allocations: 13.797 GiB, 22.07% gc time)


0.1037700673086007

In [38]:
df = spuds_to_df(setA)
df[:, :w] = rf.w1
println(df[sortperm(-rf.w1), :][1:20, :])

20×9 DataFrame
 Row │ name    h      f      l      p      r      s      a1     w          
     │ String  Int64  Int64  Int64  Int64  Int64  Int64  Int64  Float64    
─────┼─────────────────────────────────────────────────────────────────────
   1 │ spud        8      5      1      6      6      4    999  0.0065973
   2 │ spud        5      2      8      6      5      6    999  0.0064309
   3 │ spud        8      4      2      4      8      8    999  0.00634281
   4 │ spud        7      1      6      6      2      6    999  0.00588276
   5 │ spud        9      4      2      1      8      8    999  0.0058436
   6 │ spud        4      9      3      3      7      3    999  0.00562826
   7 │ spud        6      4      4      2      6      8    999  0.00549122
   8 │ spud        9      3      4      1      9      5    999  0.00547165
   9 │ spud        2      7      5      5      5      6    999  0.00537377
  10 │ spud        3      6      5      1      9      6    999  0.00526609
  11 │ spu

In [39]:
df = spuds_to_df(setB)
df[:, :w] = rf.w2
println(df[sortperm(-rf.w2), :][1:20, :])

20×9 DataFrame
 Row │ name    h      f      l      p      r      s      a1     w          
     │ String  Int64  Int64  Int64  Int64  Int64  Int64  Int64  Float64    
─────┼─────────────────────────────────────────────────────────────────────
   1 │ spud        9      4      3      7      1      4    999  0.00807099
   2 │ spud        5      6      8      2      4      2    999  0.00782641
   3 │ spud        5      1      8      3      2      8    999  0.00753292
   4 │ spud        8      4      2      3      6      7    999  0.00613395
   5 │ spud        2      8      6      1      8      5    999  0.00593829
   6 │ spud        6      1      7      2      8      3    999  0.00590894
   7 │ spud        3      7      1      6      3      6    999  0.00582089
   8 │ spud        8      4      4      2      9      2    999  0.0057035
   9 │ spud        1      4      8      9      7      8    999  0.00566436
  10 │ spud        6      5      1      7      5      5    999  0.00554697
  11 │ s

In [30]:
setA = get_spuds_with_value(scores, 100)
setB = get_spuds_with_value(scores, 95)
@time rf = ffp2(setA, setB, 100000);
rf.v

  4.325736 seconds (71.79 M allocations: 13.765 GiB, 21.75% gc time)


0.05070903260299929

In [40]:
setA = get_spuds_with_value(scores, 100)
setB = get_spuds_with_value(scores, 80)
@time rf = ffp2(setA, setB, 100000);
rf.v

  4.384748 seconds (73.84 M allocations: 14.045 GiB, 20.98% gc time)


0.21293981993245564

In [41]:
setA = get_spuds_with_value(scores, 90)
setB = get_spuds_with_value(scores, 80)
@time rf = ffp2(setA, setB, 100000);
rf.v

  4.475329 seconds (75.76 M allocations: 14.299 GiB, 21.18% gc time)


0.11376641711383198

In [42]:
setA = get_spuds_with_value(scores, 90)
setB = get_spuds_with_value(scores, 85)
@time rf = ffp2(setA, setB, 100000);
rf.v

  4.435088 seconds (75.26 M allocations: 14.232 GiB, 20.61% gc time)


0.05511890657614109

In [43]:
setA = get_spuds_with_value(scores, 92)
setB = get_spuds_with_value(scores, 88)
@time rf = ffp2(setA, setB, 100000);
rf.v

  4.490807 seconds (76.87 M allocations: 14.450 GiB, 20.26% gc time)


0.043252058226426124

In [44]:
setA = get_spuds_with_value(scores, 82)
setB = get_spuds_with_value(scores, 78)
@time rf = ffp2(setA, setB, 100000);
rf.v

  4.518490 seconds (78.26 M allocations: 14.632 GiB, 20.43% gc time)


0.047295715833653225

In [45]:
setA = get_spuds_with_value(scores, 72)
setB = get_spuds_with_value(scores, 68)
@time rf = ffp2(setA, setB, 100000);
rf.v

  4.490150 seconds (77.82 M allocations: 14.570 GiB, 20.01% gc time)


0.053791816192003426

In [46]:
setA = get_spuds_with_value(scores, 62)
setB = get_spuds_with_value(scores, 58)
@time rf = ffp2(setA, setB, 100000);
rf.v

  4.937574 seconds (88.66 M allocations: 16.014 GiB, 19.50% gc time)


0.054492030235373606

In [47]:
df = spuds_to_df(setA)
df[:, :w] = rf.w1
println(df[sortperm(-rf.w1), :][1:20, :])

20×9 DataFrame
 Row │ name    h      f      l      p      r      s      a1     w          
     │ String  Int64  Int64  Int64  Int64  Int64  Int64  Int64  Float64    
─────┼─────────────────────────────────────────────────────────────────────
   1 │ spud        2      5      5      3      5      8    999  0.0118974
   2 │ spud        7      5      2      3      3      3    999  0.0110776
   3 │ spud        7      5      1      4      2      4    999  0.0103651
   4 │ spud        7      2      4      5      8      3    999  0.0103163
   5 │ spud        3      6      5      7      5      2    999  0.0101699
   6 │ spud        1      5      6      6      6      5    999  0.0100626
   7 │ spud        5      3      5      8      3      5    999  0.00993568
   8 │ spud        6      3      5      2      5      4    999  0.00989664
   9 │ spud        7      1      4      8      1      7    999  0.00970144
  10 │ spud        5      7      2      2      6      2    999  0.00962336
  11 │ spud  

In [48]:
df = spuds_to_df(setB)
df[:, :w] = rf.w2
println(df[sortperm(-rf.w2), :][1:20, :])

20×9 DataFrame
 Row │ name    h      f      l      p      r      s      a1     w          
     │ String  Int64  Int64  Int64  Int64  Int64  Int64  Int64  Float64    
─────┼─────────────────────────────────────────────────────────────────────
   1 │ spud        6      6      1      1      1      4    999  0.0211991
   2 │ spud        2      6      5      3      6      5    999  0.019334
   3 │ spud        5      6      2      1      1      8    999  0.0176057
   4 │ spud        7      5      1      1      1      5    999  0.0150474
   5 │ spud        6      1      5      8      7      2    999  0.0140807
   6 │ spud        5      1      5      4      6      8    999  0.0132897
   7 │ spud        5      2      6      8      2      4    999  0.01162
   8 │ spud        2      5      6      6      1      7    999  0.0115321
   9 │ spud        5      3      5      8      2      5    999  0.0109364
  10 │ spud        7      5      2      2      2      3    999  0.0107704
  11 │ spud        6